In [1]:
from flask import Flask, Response
import socket
import numpy as np

In [2]:
app = Flask(__name__)


In [3]:
UDP_IP = "127.0.0.1"  # Update this with the sender's IP address
UDP_PORT = 22345  # Update this with the sender's UDP port

In [4]:
udp_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)

In [5]:
try:
    udp_socket.bind((UDP_IP, UDP_PORT))
except Exception as error:
    print(error)
    udp_socket.shutdown(1)
    udp_socket.close()
    exit(1)

In [6]:
def receive_frames():
    while True:
        try:
          data, _ = udp_socket.recvfrom(54* 96* 4)
          print(len(data))
          image = np.frombuffer(data, dtype=np.uint8).reshape(54, 96, 4)

          # Encode the gray square as a JPEG image
          _, frame_bytes = cv2.imencode('.jpg', image)

          yield (b'--frame\r\n'
                b'Content-Type: image/jpeg\r\n\r\n' + frame_bytes.tobytes() + b'\r\n')
        except KeyboardInterrupt:
            if udp_socket:
              udp_socket.close()
              exit(2)



In [7]:
@app.route('/video_feed')
def video_feed():
    return Response(receive_frames(), mimetype='multipart/x-mixed-replace; boundary=frame')



In [2]:
@app.route('/')
def index():
    image_url = "/Users/parthshah/Desktop/ML Project/python.jpeg"
    return render_template('index.html', image_url=image_url)

if __name__ == "__main__":
    app.run(host='0.0.0.0', port=9000, debug=True)
  



NameError: name 'app' is not defined